# Penman Example (ETp)

In [17]:

import ee
import openet.refetgee

ee.Initialize()


### Load GRIDMET for a single date

In [18]:
gridmet_coll = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')\
    .filterDate('2006-08-13', '2006-08-14')
    
gridmet_img = ee.Image(ee.ImageCollection(gridmet_coll.first()))
gridmet_img = ee.Image('IDAHO_EPSCOR/GRIDMET/20060813')

### Ancillary data and test point

In [19]:
test_pnt = ee.Geometry.Point(-119.01, 39.01);

# Project latitude array to match GRIDMET elevation grid exactly
elev_img = ee.Image("projects/climate-engine/gridmet/elevation")
lat_img = ee.Image.pixelLonLat().select('latitude')\
    .reproject('EPSG:4326', elev_img.projection().getInfo()['transform'])
    
# Set the output crs and crsTransform to match the GRIDMET images
crs = gridmet_img.projection().getInfo()['wkt']
geo = gridmet_img.projection().getInfo()['transform']

### Precomputed ETr

In [20]:
etr = gridmet_img.select('etr').reduceRegion(
    ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)
print(etr.getInfo()['etr'])

9.670740127563477


### Precomputed ETo

In [21]:
eto = gridmet_img.select('eto').reduceRegion(
    ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)
print(eto.getInfo()['eto'])

6.886107921600342


### Compute ETp for a single image

In [22]:
print(openet.refetgee.Daily.gridmet(gridmet_img, elev=elev_img, lat=lat_img).etp\
      .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)\
      .getInfo()['etp'])

7.623743804704581


### Compute ETp for a collection in a mapped function

In [23]:
def gridmet_etp(img):
    return ee.Image(openet.refetgee.Daily.gridmet(img, elev=elev_img, lat=lat_img).etp)\
        .copyProperties(img, ['system:time_start'])
etp = ee.Image(ee.ImageCollection(gridmet_coll.map(gridmet_etp)).first())\
    .reduceRegion(ee.Reducer.first(), test_pnt, crs=crs, crsTransform=geo)
print(etp.getInfo()['etp'])

7.623743804704581
